# Definition of a sparse matrix

In numerical analysis and scientific computing, a sparse matrix or sparse array is a matrix in which most of the elements are zero. There is no strict definition regarding the proportion of zero-value elements for a matrix to qualify as sparse but a common criterion is that the number of non-zero elements is roughly equal to the number of rows or columns. By contrast, if most of the elements are non-zero, the matrix is considered dense. The number of zero-valued elements divided by the total number of elements (e.g., m × n for an m × n matrix) is sometimes referred to as the sparsity of the matrix. (From Wikipedia)

# Applications of sparse matrices

Sparse matrices appear in the mathematical formulation of many problems, and when they do, having the correct way to represent them can be very useful to improve the efficiency of the computations:

## Pagerank

The world wide web can be represented as an adjency matrix M: if the page $i$ has a link going to the page $j$ the element $M_{ij}$ will be 1, otherwise it will be 0. This matrix is clearly extremely sparse, as the probability that a random webpage links to another random webpage is close to 0. Wikipedia has an excellent article describing how finding the "pagerank", that is, the relative importance of webpages, is analogous to finding the eigenvector with the highest eigenvalue, a problem that can be solved with repeated matrix vector multiplication (see last section of this notebook), this multiplication can be optimized if the matrix is sparse by using a correct data structure.

This is a particularly important application of sparse matrices, as trying to store and utilize the whole matrix (including the zeros) would be impossible, given the size of World Wide Web. 

## SLAM

SLAM (Simulataneous Location and Mapping), is a technique used in robotics and computer vision to estimate the pose (position and orientation) of a robot or a camera in an unknown environment, while at the same time creating a map of the environment. It is useful to build a Jacabian matrix, that contains the derivatives of all the observations with respect to all the state variables. In some cases, such as when the robot has many cameras, this matrix will be sparse, as moving camera number 4 will not change what camera number 1 sees. Knowing this, the algorithm can be optimized by representing the matrix with the correct data structure.

This is also an important application of sparse matrices as optimizing this algorithm is very important as a SLAM algorithm should be fast enough to run in real time in order to be useful.

# Implementation of a sparse matrix data structure

I will now show the implementation of one of the simplest ways to represent a sparse matrix, the "Dictionary of keys (DOK)" data structure. It consists of a dictionary that maps (row, column)-pairs to the value of the elements. This representation is simple to construct, but not the most efficient when performing operations such as matrix-vector multiplication, but still, if the matrix is really large, even this simple data structure will show significant improvements over the standard dense matrix representation.

The key idea is to completely ignore elements that are equal to zero, rather than waste time performing a huge number of multiplications by zero (that we already know will give zero as a result and not modify the final result).

In fact the algorithmic time complexity of the multiplication of a dense matrix of size NxN by a vector of size N is O(N^3), if instead we use this data structure to represent an NxN matrix that has K non-zero elements, and multiply it by a vector fof size N, the algorithmic time complexity is O(KN), that is, while using a standard dense format to represent the matrix would waste time on the zeros, by using this data structure we only use time when we encounter a non-zero element, as the elements equal to 0 are not even saved in the data structure.

In [9]:
from re import X
from dataclasses import dataclass
from pprint import pprint
from collections import OrderedDict
import numpy as np


class SparseMatrix:

    # Maybe it is better to have a dict {position: value}
    # or a dict of dicts {row_number: {column_number : value}}
    def __init__(self, items, dense_shape=None):
        # dictionary where 
        self.items = items
        self.dense_shape = dense_shape

    @staticmethod
    def from_dense(dense):
        new = SparseMatrix(OrderedDict(), dense.shape)
        for y, row in enumerate(dense):
            for x, item in enumerate(row):
                if item != 0:
                    new.items[(y, x)] = item
        return new

    def multiply_by_vector(self, vector, verbose=False):
        res = np.zeros(vector.shape)
        for (y, x), matrix_item in self.items.items():
            vector_current_item = vector[x]
            res[y] += matrix_item * vector_current_item
            if verbose:
                pprint(locals())
        return res

    def dense_representation(self):
        dense = np.zeros(self.dense_shape)
        for (y, x), item in self.items.items():
                dense[y][x] = item
        return dense

    @staticmethod
    def random_sparse_matrix_dense_repr(size, sparsity=0.5):
        matrix = np.random.rand(size, size)
        for _ in range(int(sparsity * size * size)):
            x, y = np.random.randint(0, size, size=2)
            matrix[(y,x)] = 0.0
        return matrix

    def __repr__(self):
        return f"SparseMatrix({self.items})"


    def __eq__(self, other):
        return self.items == other.items

# Examples and tests

In [10]:
m = SparseMatrix.from_dense(np.array([
    [1, 2],
    [0, 3]
]))

#print(m)
res = m.multiply_by_vector(np.array([1,2]), verbose=True)
#print(res)

{'matrix_item': 1,
 'res': array([1., 0.]),
 'self': SparseMatrix(OrderedDict([((0, 0), 1), ((0, 1), 2), ((1, 1), 3)])),
 'vector': array([1, 2]),
 'vector_current_item': 1,
 'verbose': True,
 'x': 0,
 'y': 0}
{'matrix_item': 2,
 'res': array([5., 0.]),
 'self': SparseMatrix(OrderedDict([((0, 0), 1), ((0, 1), 2), ((1, 1), 3)])),
 'vector': array([1, 2]),
 'vector_current_item': 2,
 'verbose': True,
 'x': 1,
 'y': 0}
{'matrix_item': 3,
 'res': array([5., 6.]),
 'self': SparseMatrix(OrderedDict([((0, 0), 1), ((0, 1), 2), ((1, 1), 3)])),
 'vector': array([1, 2]),
 'vector_current_item': 2,
 'verbose': True,
 'x': 1,
 'y': 1}


In [11]:
SparseMatrix.random_sparse_matrix_dense_repr(5)

array([[0.79153357, 0.93667244, 0.76867383, 0.24554535, 0.29009017],
       [0.        , 0.        , 0.64663625, 0.        , 0.        ],
       [0.29323893, 0.37634697, 0.69547036, 0.48899589, 0.        ],
       [0.        , 0.        , 0.39943284, 0.53132024, 0.        ],
       [0.        , 0.69622971, 0.        , 0.93114841, 0.        ]])

In [12]:
import numpy as np

import unittest

class TestSparseMatrix(unittest.TestCase):

    def test_simple_conversion(self):
        matrix = np.array( [
            [1, 2, 3],
            [0, 0, 0],
            [0, 0, 1]
        ])
        sparse = SparseMatrix.from_dense(matrix)
        res = SparseMatrix(OrderedDict([
            ((0, 0), 1),
            ((0, 1), 2),
            ((0, 2), 3),
            ((2, 2), 1),
        ]))
        #print(sparse, res)
        self.assertEqual(sparse, res)

    def test_simple_mult(self):
        matrix = np.array( [
            [1, 2, 3],
            [4, 0, 0],
            [0, 0, 1]
        ])

        vector = np.array( [
            1,
            2,
            3
        ])

        numpy_result = matrix @ vector

        sparse_matrix = SparseMatrix.from_dense(matrix)
        res = np.array([1*1+2*2+3*3,
                        1*4,
                        3*1
                        ])
        sparse_res = sparse_matrix.multiply_by_vector(vector)

        #print(locals())
        np.testing.assert_array_almost_equal(res, numpy_result) 
        np.testing.assert_array_almost_equal(res, sparse_res) 

    def test_back_and_forth_is_equal_to_start(self):
        for _ in range(20):
            matrix = SparseMatrix.random_sparse_matrix_dense_repr(20)
            sparse = SparseMatrix.from_dense(matrix)
            dense_back = sparse.dense_representation()
            np.testing.assert_array_almost_equal(matrix, dense_back)

    
    def test_multiplication_is_correct_oracle(self):
        for _ in range(20):

            matrix = SparseMatrix.random_sparse_matrix_dense_repr(5)
            vector = np.random.rand(5)

            numpy_result = matrix @ vector

            sparse_matrix = SparseMatrix.from_dense(matrix)
            res = sparse_matrix.multiply_by_vector(vector)

            #pprint(locals())
            np.testing.assert_array_almost_equal(res, numpy_result)
    
    

unittest.main(verbosity=3, argv=['first-arg-is-ignored'], exit=False)

test_back_and_forth_is_equal_to_start (__main__.TestSparseMatrix) ... ok
test_multiplication_is_correct_oracle (__main__.TestSparseMatrix) ... ok
test_simple_conversion (__main__.TestSparseMatrix) ... ok
test_simple_mult (__main__.TestSparseMatrix) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.092s

OK


# Finding eigenvectors with repeated multiplication

When a vector is multiplied by a matrix that has at least one eigenvector, the component of the vector that is parallel to the eigenvector with the maximal eigenvalue (in module) is stretched by the eigenvalue of that eigenvector. The perpendicular components are modified in other ways, such as being rotated, deformed or stretched **less**. Iterating the alternation of this product and a normalization operation on the vector, all the other components of the vector perpendicular to the eigenvector with maximal eigenvalue will be removed and it will become equal to the eigenvector with the maximal eigenvalue. This is important for the PageRank algorithm, I suggest the excellent English Wikipedia article on PageRank for further reading.

In [13]:
matrix = np.random.rand(3,3)
v = np.random.rand(3)

values, vectors = np.linalg.eig(matrix)
eig = vectors[:, 0]

print("Note: an error of 2 is like an error of 0: the vectors just point in opposite directions (v_1 = -v_2)")

for _ in range(15):
    print(f"Current estimate {v}, error from eigenvector {np.linalg.norm(v - eig)}")
    r = matrix @ v
    v = r / np.linalg.norm(r)

values, vectors = np.linalg.eig(matrix)
print(vectors[:, 0])

Note: an error of 2 is like an error of 0: the vectors just point in opposite directions (v_1 = -v_2)
Current estimate [0.97841236 0.50677034 0.93098361], error from eigenvector 2.393000736292868
Current estimate [0.31257968 0.54548737 0.77764868], error from eigenvector 1.9992461461742412
Current estimate [0.33521081 0.48640987 0.80686997], error from eigenvector 1.9999446980688422
Current estimate [0.33091184 0.50277569 0.79856995], error from eigenvector 1.9999960511017525
Current estimate [0.33203959 0.49842065 0.80082867], error from eigenvector 1.999999719351945
Current estimate [0.33174038 0.49958294 0.80022822], error from eigenvector 1.9999999800507444
Current estimate [0.3318202  0.49927315 0.80038845], error from eigenvector 1.9999999985821337
Current estimate [0.33179892 0.49935575 0.80034575], error from eigenvector 1.9999999998992242
Current estimate [0.33180459 0.49933373 0.80035713], error from eigenvector 1.9999999999928373
Current estimate [0.33180308 0.4993396  0.800